### Q: is the champion easy or difficult to master? 

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
import pandas as pd
import tensorflow as tf

# Import our input dataset
mastery_df = pd.read_csv('https://raw.githubusercontent.com/frlinh/game-analysis/main/championMasteryBeta.csv')
mastery_df.head()

,Date,championId,championLevel,championPoints,lastPlayTime,championPointsSinceLastLevel,championPointsUntilNextLevel,chestGranted,tokensEarned,summonerId
0,8/27/2022,350,7,136999,1661483549000,115399,0,True,0,vBrYfjxI-dF_QLzo6JnZfJCIBs02iRWw2iJUqPOjVjCCWNRb
1,8/27/2022,117,7,113568,1661644216000,91968,0,True,0,vBrYfjxI-dF_QLzo6JnZfJCIBs02iRWw2iJUqPOjVjCCWNRb
2,8/27/2022,143,7,100598,1660524764000,78998,0,True,0,vBrYfjxI-dF_QLzo6JnZfJCIBs02iRWw2iJUqPOjVjCCWNRb
3,8/27/2022,25,6,90688,1660354753000,69088,0,False,0,vBrYfjxI-dF_QLzo6JnZfJCIBs02iRWw2iJUqPOjVjCCWNRb
4,8/27/2022,147,6,85820,1660539194000,64220,0,True,1,vBrYfjxI-dF_QLzo6JnZfJCIBs02iRWw2iJUqPOjVjCCWNRb


# Logistic Regression
Classification algorithm that can analyze continuous and categorical variables. Predicts the probability of the input data belonging to one of two groups
* can be overwhelmed by too much data

In [2]:
cleaned_mastery_df = mastery_df.drop(columns=['Date', 'lastPlayTime', 'championPointsSinceLastLevel', 'championPointsUntilNextLevel', 'chestGranted', 'chestGranted', 'summonerId'])
# cleaned_mastery_df = mastery_df.drop(columns='lastPlayTime')
# cleaned_mastery_df = mastery_df.drop(columns='championPointsSinceLastLevel')
# mastery_df = mastery_df.drop(columns='championPointsUntilNextLevel')
# mastery_df = mastery_df.drop(columns='chestGranted')
# mastery_df = mastery_df.drop(columns='summonerId')
cleaned_mastery_df

,championId,championLevel,championPoints,tokensEarned
0,350,7,136999,0
1,117,7,113568,0
2,143,7,100598,0
3,25,6,90688,0
4,147,6,85820,1
...,...,...,...,...
8792,57,1,193,0
8793,200,1,171,0
8794,268,1,164,0
8795,37,1,154,0


In [3]:
cleaned_mastery_df.dtypes

championId        int64
championLevel     int64
championPoints    int64
tokensEarned      int64
dtype: object

In [4]:
cleaned_mastery_df.describe()

,championId,championLevel,championPoints,tokensEarned
count,8797.000000,8797.000000,8.797000e+03,8797.000000
mean,144.514266,2.571217,1.626137e+04,0.105718
std,176.282548,1.728392,6.370754e+04,0.440359
min,1.000000,1.000000,6.600000e+01,0.000000
25%,37.000000,1.000000,1.035000e+03,0.000000
50%,85.000000,2.000000,3.713000e+03,0.000000
75%,157.000000,3.000000,1.182900e+04,0.000000
max,895.000000,7.000000,2.056376e+06,3.000000


In [5]:
#define x and y
X = cleaned_mastery_df.drop(columns='championLevel')
y= cleaned_mastery_df.championLevel

#split training/test data
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=1, stratify=y)

In [6]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [7]:
# Define the logistic regression model
log_classifier = LogisticRegression(solver="lbfgs",max_iter=200)

# Train the model
log_classifier.fit(X_train,y_train)

# Evaluate the model
y_pred = log_classifier.predict(X_test)
print(f" Logistic regression model accuracy: {accuracy_score(y_test,y_pred):.3f}")

 Logistic regression model accuracy: 0.446


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


# vs. Basic Neural Network
* prone to overfitting and can be more difficult to train than a logistic regression model 
* thrives with a lot of data

In [8]:
# Define the basic neural network model
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=16, activation="relu", input_dim=3))
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50)

# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/50
207/207 [==============================] - 2s 4ms/step - loss: -0.6477 - accuracy: 0.3453
Epoch 2/50
207/207 [==============================] - 1s 3ms/step - loss: -5.2122 - accuracy: 0.3524
Epoch 3/50
207/207 [==============================] - 1s 4ms/step - loss: -13.5349 - accuracy: 0.3524
Epoch 4/50
207/207 [==============================] - 1s 4ms/step - loss: -26.9757 - accuracy: 0.3524
Epoch 5/50
207/207 [==============================] - 1s 3ms/step - loss: -46.1920 - accuracy: 0.3524
Epoch 6/50
207/207 [==============================] - 1s 4ms/step - loss: -70.2401 - accuracy: 0.3524
Epoch 7/50
207/207 [==============================] - 1s 3ms/step - loss: -98.2380 - accuracy: 0.3524
Epoch 8/50
207/207 [==============================] - 1s 3ms/step - loss: -129.9086 - accuracy: 0.3524
Epoch 9/50
207/207 [==============================] - 1s 4ms/step - loss: -165.1861 - accuracy: 0.3524
Epoch 10/50
207/207 [==============================] - 1s 3ms/step - loss: -203.63

# Support Vector Machine (SVM)
binary classifier that distinguishes data points from 2 separate groups. tries to maximize the distance between the 2 closest data points of the groups.
* can build adequate models with linear/nonlinear data unlike logistic regression
* can analyze and interpret multiple data types
* less prone to overfitting 

In [9]:
#check the # of unique items in each column
cleaned_mastery_df.nunique()

#bucket championLevel

#bucket championPoints

#bucket lastPlayTime

#bucket championPointsSinceLastLevel

#bucket championPointsUntilNextLevel

championId         161
championLevel        7
championPoints    6288
tokensEarned         4
dtype: int64

In [10]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder 
encode_df = pd.DataFrame(enc.fit_transform(mastery_df))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(mastery_df)
encode_df.head()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


ValueError: ignored

In [ ]:
#merge the new encode_df with the diamondPlayerIDs dataset?

In [ ]:
# define x and y
y = 
X = 

# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
# Create the SVM model
svm = SVC(kernel='linear')

In [ ]:
# Train the model
svm.fit(X_train, y_train)

In [ ]:
# Evaluate the model
y_pred = svm.predict(X_test_scaled)
print(f" SVM model accuracy: {accuracy_score(y_test,y_pred):.3f}")

# vs Deep Learning Model

In [ ]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 10
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Train the model 
fit_model = nn.fit(X_train_scaled, y_train, epochs=50) 
# Evaluate the model using the test data 
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

# Random Forest

In [ ]:
#import dataset
mastery_df = pd.read_csv('championMasteryBeta.csv')
mastery_df.head()

#drop the columns we don't need
mastery_df = mastery_df.drop(columns='Date')
mastery_df = mastery_df.drop(columns='tokensEarned')
mastery_df = mastery_df.drop(columns='chestGranted')
mastery_df = mastery_df.drop(columns='summonerId') #most likely keep this later on to merge our diamondPlayer dataset with this one 
mastery_df

In [ ]:
#check the # of unique items in each column
mastery_df.nunique()

#bucket championLevel

#bucket championPoints

#bucket lastPlayTime

#bucket championPointsSinceLastLevel

#bucket championPointsUntilNextLevel

In [ ]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder 
encode_df = pd.DataFrame(enc.fit_transform(mastery_df))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(mastery_df)
encode_df.head()

In [ ]:
#merge the new encode_df with the diamondPlayerIDs dataset?

In [ ]:
#define x and y
y = 
X = 

# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=128, random_state=78)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred = rf_model.predict(X_test_scaled)
print(f" Random forest predictive accuracy: {accuracy_score(y_test,y_pred):.3f}")

# vs Deep Learning

In [ ]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 24
hidden_nodes_layer2 = 12

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")